In [1]:
import pandas as pd

In [2]:
text = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="text")
form = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="form")

In [3]:
from nltk.corpus import stopwords

# Filter the form to not include stopwords for comparing to the original form later on
filtered = form.loc[form.loc[:, "Word"].apply(lambda x: x not in stopwords.words("english"))]

In [4]:
# Original text and form
og_text = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\unigram_data.h5", key="text")
og_form = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\unigram_data.h5", key="form")

In [5]:
# Melt the text into long format one row for each review
og_text_melt = og_text.melt(id_vars=["Artist", "Album"], value_vars=["Pitchfork", "Guardian", "Spectrum", "NME"],
                            var_name="Platform", value_name="Text")

In [6]:
# Explode to word level and sort by album, artist, platform
og_text_melt = og_text_melt.explode("Text").sort_values(by=["Album", "Artist", "Platform"]).reset_index(drop=True)
# Sort by album, artist, platform
og_form = og_form.sort_values(by=["Album", "Artist", "Platform"]).reset_index(drop=True)

In [7]:
# Join the form and the text together
og_merged = pd.concat([og_form, og_text_melt.loc[:, "Text"]], axis=1)

In [9]:
# Copy of og_text_melt essentially but keeping full text
review_ids = og_text.melt(id_vars=["Artist", "Album"], value_vars=["Pitchfork", "Guardian", "Spectrum", "NME"],
                          var_name="Platform", value_name="Text").sort_values(
    by=["Album", "Artist", "Platform"]).reset_index(drop=True)
# Add review ids
review_ids.loc[:, "Review id"] = [x for x in range(review_ids.shape[0])]

In [10]:
# Join the text together for duplication reasons
review_ids.loc[:, "Text"] = review_ids.loc[:, "Text"].apply(lambda x: "|".join(x))

In [11]:
# Get whether each review is duplicated
review_ids.loc[:, "Dupes"] = review_ids.duplicated(subset=["Text"])

In [12]:
# Keep the second version of each duplicated review
dupes = review_ids.drop_duplicates(subset=["Text"], keep="last")
# Now reduce to only those that are duplicates
dupes = dupes.loc[dupes.loc[:, "Dupes"]]

In [13]:
# Fill non-dupes with false
review_ids.loc[:, "Dupes"] = dupes
review_ids = review_ids.fillna(False)

In [15]:
review_ids.loc[:, "Text"] = review_ids.loc[:, "Text"].apply(lambda x: x.split("|"))
review_ids = review_ids.explode("Text").reset_index(drop=True)

In [16]:
og_merged = og_merged.loc[~review_ids.loc[:, "Dupes"]]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [18]:
review_ids

,Artist,Album,Platform,Text,Review id,Dupes
0,beyonce,4,Guardian,Beyoncé's,0,False
1,beyonce,4,Guardian,new,0,False
2,beyonce,4,Guardian,album,0,False
3,beyonce,4,Guardian,bad,0,False
4,beyonce,4,Guardian,game-changer,0,False
...,...,...,...,...,...,...
289934,marvin gaye,you're the man,Spectrum,masterpiece,751,False
289935,marvin gaye,you're the man,Spectrum,it’s,751,False
289936,marvin gaye,you're the man,Spectrum,fun,751,False
289937,marvin gaye,you're the man,Spectrum,imagine,751,False


In [ ]:
og_merged = og_merged.reset_index(drop=True)

In [ ]:
pd.set_option('display.max_rows', 10)

og_merged.loc[og_merged["Desired"] == 1]

In [ ]:
og_merged = og_merged.sort_values(by=["Artist", "Album", "Platform"]).reset_index(drop=True)

In [ ]:
og_form = og_form.sort_values(by=["Artist", "Album", "Platform"]).reset_index(drop=True)

In [ ]:
og_form.loc[:, "Desired"] = og_merged.loc[:, "Desired"]

In [ ]:
og_form = og_form.fillna(-1)

In [ ]:
text = form.loc[:, "Word"]
form.loc[:, "Word"] = form.loc[:, "Word"].apply(lambda x: x.lower())
test = pd.merge(form.drop(columns=["Desired"]), og_merged.rename(
    {"id": "Word id", "Text": "Word"}, axis=1), 
                on=["Artist", "Album", "Platform", "Word id", "Word"], how="left").fillna(-1)

,Artist,Album,Platform,id,Desired,Text
0,sleaford mods,key markets,Pitchfork,0,-1.0,The
1,sleaford mods,key markets,Pitchfork,1,-1.0,title
2,sleaford mods,key markets,Pitchfork,2,-1.0,Key
3,sleaford mods,key markets,Pitchfork,3,-1.0,Markets—
4,sleaford mods,key markets,Pitchfork,4,-1.0,something
...,...,...,...,...,...,...
273593,bjork,fossora,NME,476,-1.0,Emily
273594,bjork,fossora,NME,477,-1.0,Mackay
273595,bjork,fossora,NME,478,-1.0,29th
273596,bjork,fossora,NME,479,-1.0,September


In [608]:
artist_album_sort = form.drop_duplicates(subset=["Artist", "Album", "Platform"])
og_merged.loc[:, "Artist"] = pd.Categorical(og_merged.loc[:, "Artist"], artist_album_sort.loc[:, "Artist"].unique())
og_merged.loc[:, "Album"] = pd.Categorical(og_merged.loc[:, "Album"], artist_album_sort.loc[:, "Album"].unique())
og_merged.loc[:, "Platform"] = pd.Categorical(og_merged.loc[:, "Platform"], ["Pitchfork", "Guardian", "Spectrum", "NME"])
og_merged = og_merged.sort_values(["Platform", "Album", "Artist"])

In [609]:
og_merged = og_merged.reset_index(drop=True)

In [610]:
pd.set_option('display.max_rows', 10)
og_merged.loc[(og_merged["Artist"] == "blood orange") & (og_merged["Platform"] == "Guardian")]

,Artist,Album,Platform,id,Desired,Text
97829,blood orange,negro swan,Guardian,0,-1.0,British-born
97830,blood orange,negro swan,Guardian,1,-1.0,musician
97831,blood orange,negro swan,Guardian,2,-1.0,Devonté
97832,blood orange,negro swan,Guardian,3,-1.0,Hynes
97833,blood orange,negro swan,Guardian,4,-1.0,quietly
...,...,...,...,...,...,...
98304,blood orange,negro swan,Guardian,299,-1.0,weirdly
98305,blood orange,negro swan,Guardian,300,-1.0,unmoored
98306,blood orange,negro swan,Guardian,301,-1.0,worse
98307,blood orange,negro swan,Guardian,302,-1.0,emotionally


In [611]:
filtered.loc[(filtered["Artist"] == "blood orange") & (filtered["Platform"] == "Guardian")]

,Artist,Album,Platform,Review id,Word id,Word,Desired
158941,blood orange,negro swan,Guardian,179,0,British-born,-1
158942,blood orange,negro swan,Guardian,179,1,musician,-1
158943,blood orange,negro swan,Guardian,179,2,Devonté,-1
158944,blood orange,negro swan,Guardian,179,3,Hynes,-1
158946,blood orange,negro swan,Guardian,179,5,quietly,-1
...,...,...,...,...,...,...,...
194526,blood orange,negro swan,Guardian,254,448,weirdly,-1
194527,blood orange,negro swan,Guardian,254,449,unmoored,-1
194531,blood orange,negro swan,Guardian,254,453,worse,-1
194532,blood orange,negro swan,Guardian,254,454,emotionally,-1


In [612]:
filtered.loc[:, ["Artist", "Album"]].drop_duplicates()

,Artist,Album
0,sleaford mods,key markets
627,black midi,hellfire
1609,kanye west,yeezus
3223,kehlani,it was good until it wasn't
3880,blood orange,negro swan
...,...,...
152084,father john misty,fear fun
152751,beyonce,4
153820,toro y moi,anything in return
154870,james blake,assume form


In [ ]:
pd.set_option("display.max_rows", 10)
og_merged.loc[(og_merged["Artist"] == "blood orange") & (og_merged["Platform"] == "Guardian")]

In [614]:
filtered.loc[:, "Desired"] = og_merged.loc[:, "Desired"].tolist()

C:\Users\tommy\AppData\Local\Temp\ipykernel_3796\2352118031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.loc[:, "Desired"] = og_merged.loc[:, "Desired"].tolist()


In [615]:
filtered.loc[filtered["Desired"] == 1]

,Artist,Album,Platform,Review id,Word id,Word,Desired
301,sleaford mods,key markets,Pitchfork,0,301,working,1.0
302,sleaford mods,key markets,Pitchfork,0,302,class,1.0
304,sleaford mods,key markets,Pitchfork,0,304,politics,1.0
752,black midi,hellfire,Pitchfork,1,125,characters,1.0
753,black midi,hellfire,Pitchfork,1,126,historical,1.0
978,black midi,hellfire,Pitchfork,1,351,Weimar,1.0
1491,black midi,hellfire,Pitchfork,1,864,military,1.0
1498,black midi,hellfire,Pitchfork,1,871,theater,1.0
1759,kanye west,yeezus,Pitchfork,2,150,loyalty,1.0
1760,kanye west,yeezus,Pitchfork,2,151,respect,1.0


In [616]:
pd.set_option('display.max_rows', 10)
filtered.loc[filtered["Desired"] == 1]

,Artist,Album,Platform,Review id,Word id,Word,Desired
301,sleaford mods,key markets,Pitchfork,0,301,working,1.0
302,sleaford mods,key markets,Pitchfork,0,302,class,1.0
304,sleaford mods,key markets,Pitchfork,0,304,politics,1.0
752,black midi,hellfire,Pitchfork,1,125,characters,1.0
753,black midi,hellfire,Pitchfork,1,126,historical,1.0
...,...,...,...,...,...,...,...
438476,bjork,fossora,NME,713,418,processing,1.0
438478,bjork,fossora,NME,713,420,progressing,1.0
438620,bjork,fossora,NME,713,562,love,1.0
438705,bjork,fossora,NME,713,647,reinfatuation,1.0


In [617]:
pd.set_option("display.max_rows", 10)
filtered.loc[filtered["Desired"] == 1]

,Artist,Album,Platform,Review id,Word id,Word,Desired
301,sleaford mods,key markets,Pitchfork,0,301,working,1.0
302,sleaford mods,key markets,Pitchfork,0,302,class,1.0
304,sleaford mods,key markets,Pitchfork,0,304,politics,1.0
752,black midi,hellfire,Pitchfork,1,125,characters,1.0
753,black midi,hellfire,Pitchfork,1,126,historical,1.0
...,...,...,...,...,...,...,...
438476,bjork,fossora,NME,713,418,processing,1.0
438478,bjork,fossora,NME,713,420,progressing,1.0
438620,bjork,fossora,NME,713,562,love,1.0
438705,bjork,fossora,NME,713,647,reinfatuation,1.0


In [618]:
form.loc[:, "Desired"] = filtered.loc[:, "Desired"]

In [619]:
form.loc[:, "Desired"] = form.loc[:, "Desired"].fillna(-1)

In [620]:
form.loc[(form["Desired"] == 1) & (form["Artist"] == "blood orange") & (form["Platform"] == "Guardian")]

,Artist,Album,Platform,Review id,Word id,Word,Desired
159196,blood orange,negro swan,Guardian,179,255,black,1.0
159199,blood orange,negro swan,Guardian,179,258,ugly,1.0


In [621]:
form.loc[form["Desired"] == 1]

,Artist,Album,Platform,Review id,Word id,Word,Desired
301,sleaford mods,key markets,Pitchfork,0,301,working,1.0
302,sleaford mods,key markets,Pitchfork,0,302,class,1.0
304,sleaford mods,key markets,Pitchfork,0,304,politics,1.0
752,black midi,hellfire,Pitchfork,1,125,characters,1.0
753,black midi,hellfire,Pitchfork,1,126,historical,1.0
...,...,...,...,...,...,...,...
438476,bjork,fossora,NME,713,418,processing,1.0
438478,bjork,fossora,NME,713,420,progressing,1.0
438620,bjork,fossora,NME,713,562,love,1.0
438705,bjork,fossora,NME,713,647,reinfatuation,1.0


In [622]:
artist_album_sort = form.drop_duplicates(subset=["Artist", "Album", "Platform"])
form.loc[:, "Artist"] = pd.Categorical(form.loc[:, "Artist"], artist_album_sort.loc[:, "Artist"].unique())
form.loc[:, "Album"] = pd.Categorical(form.loc[:, "Album"], artist_album_sort.loc[:, "Album"].unique())
form.loc[:, "Platform"] = pd.Categorical(form.loc[:, "Platform"], ["Pitchfork", "Guardian", "Spectrum", "NME"])
form = form.sort_values(["Album", "Artist", "Platform"])

In [623]:
form = form.reset_index(drop=True)

In [624]:
text.loc[:, "Artist"] = pd.Categorical(text.loc[:, "Artist"], artist_album_sort.loc[:, "Artist"].unique())
text.loc[:, "Album"] = pd.Categorical(text.loc[:, "Album"], artist_album_sort.loc[:, "Album"].unique())
text.loc[:, "Platform"] = pd.Categorical(text.loc[:, "Platform"], ["Pitchfork", "Guardian", "Spectrum", "NME"])
text = text.sort_values(["Album", "Artist", "Platform"]).reset_index(drop=True)

In [625]:
text = text.astype({"Artist": object,
                    "Album": object,
                    "Platform": object,
                    "Text": object,
                    "Review id": int})

In [626]:
form = form.astype({"Artist": str,
                    "Album": str,
                    "Platform": str,
                    "Word": str})

In [ ]:
text.to_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="text", mode="a", format="t")

In [629]:
form.to_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="form", mode="a", format="t")

In [490]:
pd.set_option("display.max_rows", 1000)

In [627]:
text.loc[:, "Text"] = text.loc[:, "Text"].apply(lambda x: "|".join(x))

In [628]:
text.loc[:, "Text"] = text.loc[:, "Text"].apply(lambda x: x.split("|"))

In [631]:
text.to_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="text", mode="a")

C:\Users\tommy\AppData\Local\Temp\ipykernel_3796\2065478248.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['Artist', 'Album', 'Platform', 'Text'], dtype='object')]

  text.to_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="text", mode="a")
